# Lecture 9 – Multiple Linear Regression and Feature Engineering

## DSC 40A, Fall 2021

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

---

## Part 1 – Multiple Linear Regression

---

Here is a data set of sales figures from different stores.

In [ ]:
data = pd.read_csv('sales.csv')
data

## Using just one feature

Before we perform multiple linear regression, let's first just perform simple linear regression. We'll try and use square footage to predict net sales; our prediction rule will be

$$
\text{predicted net sales} = w_0 + w_1 (\text{square feet})
$$

In [ ]:
px.scatter(data, x='sq_ft', y='net_sales', title='Net Sales vs. Square Feet')

It seems like $w_1^*$, the optimal slope, should be positive. To find $w_0^*$ and $w_1^*$, we'll solve the normal equations.

In [ ]:
def solve_normal_equations(X, y):
    '''Returns the optimal parameter vector, w*, given a design matrix X and observation vector y.'''
    return np.linalg.solve(X.T @ X, X.T @ y)

In [ ]:
data['1'] = 1

X_one_feature_model = data[['1', 'sq_ft']]
X_one_feature_model.values

In [ ]:
y = data['net_sales']

In [ ]:
w_one_feature_model = solve_normal_equations(X_one_feature_model, y)
w_one_feature_model

This is telling us that the best-fitting line to this dataset is

$$\text{predicted net sales} = 2.577 + 85.389 (\text{square feet})$$

To get predictions for all observations in my dataset:

In [ ]:
X_one_feature_model @ w_one_feature_model

Let's draw a plot of our prediction rule.

In [ ]:
px.scatter(data, x='sq_ft', y='net_sales', title='Net Sales vs. Square Feet')

x_range = np.linspace(0, 10)

fig = go.Figure()
fig.add_trace(go.Scatter(x = data['sq_ft'], y = y, mode = 'markers', name = 'actual'))
fig.add_trace(go.Scatter(x = x_range, 
                         y = w_one_feature_model[0] + w_one_feature_model[1] * x_range, 
                         name = 'linear prediction rule', 
                         line=dict(color='red')))

fig.update_layout(xaxis_title = 'Square Feet', yaxis_title = 'Net Sales')

It's also worth calculating the mean squared error of this prediction rule, so that we can compare it to our later prediction rules.

In [ ]:
def mean_squared_error(X, y, w):
    return np.mean(np.sum((y - X @ w)**2))

## Using two features

Let's now try to predict net sales from two variables: the square footage (size) of the store, and the number of competing stores in the area. Our model will be:

$$
\text{predicted net sales} = w_0 + w_1 (\text{square feet}) + w_2(\text{competitors})
$$

Suppose $w_0^*$, $w_1^*$, and $w_2^*$ are our prediction rule's optimal parameters. Do you expect $w_1^*$ to be positive or negative? What about $w_2^*$?

In [ ]:
px.scatter(data, x='sq_ft', y='net_sales', title='Net Sales vs. Square Feet')

In [ ]:
px.scatter(data, x='competing_stores', y='net_sales', title='Net Sales vs. Competing Stores')

Looking at separate scatter plots only tells part of the story. Let's look at a 3D scatter plot, with one axis for square footage, one axis for competing stores, and one axis for net sales.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter3d(x=data['sq_ft'], 
                           y=data['competing_stores'], 
                           z=data['net_sales'], mode='markers'))

fig.update_layout(scene = dict(
    xaxis_title = 'Square Feet',
    yaxis_title = 'Competing Stores',
    zaxis_title = 'Net Sales'))

Our goal is to find the best fitting **plane** to this set of points.

### Discussion Question

At the start of this notebook, we fit a prediction rule with a single feature, square feet, and got that the weight of that feature was $w_1^* = 85.389$.

We are about to fit a prediction rule with two features, square feet and competing stores.

**Question:** Is the weight of the square feet feature, $w_1^*$, for this **new** prediction rule guaranteed to be equal to 85.389?

A. Yes

B. No

### To answer, go to [menti.com](https://menti.com) and enter the code 5115 8817.

Our design matrix is:
    
$$
\begin{pmatrix}
 1 & s_1 & c_1\\
 1 & s_2 & c_2\\
 \vdots & \vdots & \vdots\\
 1 & s_n & c_n
\end{pmatrix}
$$

where $s_i$ is the size of the $i$th store, and $c_n$ is the number of competitors. In code:

In [ ]:
X_two_feature_model = data[['1', 'sq_ft', 'competing_stores']].values
X_two_feature_model

Using the function `solve_normal_equations` that we already built:

In [ ]:
w_two_feature_model = solve_normal_equations(X_two_feature_model, y)
w_two_feature_model

This is telling us that the best-fitting plane to this dataset is

$$\text{predicted net sales} = 303.491 + 45.151 (\text{square feet}) - 21.585 (\text{competing stores})$$

**Note that the weight of $\text{square feet}$ in this prediction rule is different than the weight of $\text{square feet}$ in the prediction rule that only had one feature!**

In [ ]:
XX, YY = np.mgrid[1:10:2, 0:16:2]
Z = w_two_feature_model[0] + w_two_feature_model[1] * XX + w_two_feature_model[2] * YY
plane = go.Surface(x=XX, y=YY, z=Z)

fig = go.Figure(data=[plane])
fig.add_trace(go.Scatter3d(x=data['sq_ft'], 
                           y=data['competing_stores'], 
                           z=data['net_sales'], mode='markers', marker = {'color': '#656DF1'}))

fig.update_layout(scene = dict(
    xaxis_title = 'Square Feet',
    yaxis_title = 'Competing Stores',
    zaxis_title = 'Net Sales'))

As before, let's calculate the MSE:

In [ ]:
mean_squared_error(X_two_feature_model, y, w_two_feature_model)

Note that this is significantly lower than the MSE of the model with just one feature:

In [ ]:
mean_squared_error(X_one_feature_model, y, w_one_feature_model)

## All features

Let's fit a prediction rule using all of the features.

In [ ]:
column_order = ['1', 'sq_ft', 'competing_stores', 'inventory', 'advertising', 'district_size']
X_all_features = data[column_order].values
X_all_features

In [ ]:
w_all_features = solve_normal_equations(X_all_features, y)
w_all_features

In [ ]:
for i, feature in enumerate(column_order):
    if feature == '1':
        print(f'intercept:\t{w_all_features[0]:0.3f}')
    else:
        print(f'{feature}:\t{w_all_features[i]:0.3f}')

The MSE of this model is even lower!

In [ ]:
mean_squared_error(X_all_features, y, w_all_features)

Note that I can't visualize this prediction rule, since I would need to be able to visualize in 6D, but I can still find this prediction rule's predictions:

In [ ]:
X_all_features @ w_all_features

## Which feature is most "important"?

We should standardize in order to account for the difference in units and scale between the features.

**Question:** What would happen if I try to standardize the column of all 1s? 🧐

In [ ]:
features = data[column_order].iloc[:, 1:].values

In [ ]:
standardized_features = (features - features.mean(axis=0)) / features.std(axis=0)

In [ ]:
X_standardized = np.column_stack([
    np.ones(data.shape[0]),
    standardized_features
])

In [ ]:
w_standardized = solve_normal_equations(X_standardized, y)
w_standardized

In [ ]:
for i, feature in enumerate(column_order):
    if feature == '1':
        print(f'intercept:\t{w_standardized[0]:0.3f}')
    else:
        print(f'{feature}:\t{w_standardized[i]:0.3f}')

The district size appears to have the largest effect on the net sales.

In [ ]:
mean_squared_error(X_standardized, y, w_standardized)

Note that standardizing has no impact on the actual predictions made by our prediction rule, and hence the MSE – it just makes the weights more interpretable.

---

## Part 2: Feature Engineering

---

In [ ]:
cars = sns.load_dataset('mpg').dropna()

In [ ]:
cars

In [ ]:
px.scatter(cars, x='horsepower', y='mpg', title='MPG vs. Horsepower')

A regular linear model here isn't great.

In [ ]:
cars['1'] = 1
w_cars_one_feature = solve_normal_equations(cars[['1', 'horsepower']], cars['mpg'])
w_cars_one_feature

In [ ]:
px.scatter(cars, x='horsepower', y='mpg', title='MPG vs. Horsepower')

x_range = np.linspace(40, 220)

fig = go.Figure()
fig.add_trace(go.Scatter(x = cars['horsepower'], y = cars['mpg'], mode = 'markers', name = 'actual'))
fig.add_trace(go.Scatter(x = x_range, 
                         y = w_cars_one_feature[0] + w_cars_one_feature[1] * x_range, 
                         name = 'linear prediction rule', 
                         line=dict(color='red')))

fig.update_layout(xaxis_title = 'Horsepower', yaxis_title = 'MPG')

What if we add $\text{horsepower}^2$ as a feature?

In [ ]:
cars['horsepower^2'] = cars['horsepower']**2

In [ ]:
cars[['1', 'horsepower', 'horsepower^2']]

In [ ]:
w_cars_squared = solve_normal_equations(cars[['1', 'horsepower', 'horsepower^2']], cars['mpg'])
w_cars_squared

Let's look at the resulting prediction rule.

In [ ]:
px.scatter(cars, x='horsepower', y='mpg', title='MPG vs. Horsepower')

fig = go.Figure()
fig.add_trace(go.Scatter(x = cars['horsepower'], y = cars['mpg'], mode = 'markers', name = 'actual'))
fig.add_trace(go.Scatter(x = x_range, 
                         y = w_cars_one_feature[0] + w_cars_one_feature[1] * x_range, 
                         name = 'linear prediction rule', 
                         line=dict(color='red')))
fig.add_trace(go.Scatter(x = np.linspace(40, 220), 
                         y = w_cars_squared[0] + w_cars_squared[1] * x_range + w_cars_squared[2] * x_range**2, 
                         name = 'quadratic prediction rule', 
                         line=dict(color='#F7CF5D', width=5)))

fig.update_layout(xaxis_title = 'Horsepower', yaxis_title = 'MPG')

Note: this is **still** a linear prediction rule – it's just not linear in terms of horsepower. **It is linear in terms of the parameter vector, $\vec{w}$, which means we can still use the normal equations to find $\vec{w}^*$**.